In [45]:
import os
import sys
sys.path.append('../')  # subir 1 nivel desde notebooks a src
from src.config.settings import COUNTRY, SRC_DIR

In [46]:
from src.sources.inventory.app import Inventory
db_loader = Inventory()
df_data_base = db_loader.load_db_from_country_selected()

CO


c:\Users\JTRUJILLO\Documents\Galgo\Scripts\Otros\deep_research_models\notebooks\..\src\sources\inventory\app.py:23: DtypeWarning: Columns (8,23,30) have mixed types. Specify dtype option on import or set low_memory=False.
  data_base = pd.read_csv(CO_INVENTORY_PATH)


In [ ]:
models_list = [
    "CO731-tvs-raider-125",
    "CO928-akt-motos-nkd-125",
    "CO2961-hero-hunk-125-r",
    "CO2861-tvs-raider-fi-125",
    "CO1033-bajaj-boxer-ct-100-es",
    "CO1996-hero-hunk-160r-4v",
    "CO736-tvs-apache-rtr-160-4v-xc-fi",
    "CO734-tvs-apache-rtr-160-4v-xconnect",
    "CO1480-hero-eco-deluxe-cw-c-2024",
    "CO569-tvs-neo-nx",
    "CO570-tvs-raider-125-racing" # Problemática
]

In [ ]:
from src.utils.datetime_utils import get_current_date_format_yyyy_mm_dd
from src.core.prompt_utils import *
from src.core.gemini_processor import GeminiProcessor
gemini_processor = GeminiProcessor()

In [49]:
for model in models_list:
    df_modelo = df_data_base[df_data_base["code"] == model]
    data_model = get_basic_data_model(df_modelo, COUNTRY)

    basic_data_model_info = f"""
[DATOS BASICOS]
Marca: {data_model["marca"]}
Modelo: {data_model["modelo"]}
Año: {data_model["año"]}
Pais: {data_model["pais"]}
Tipo: {data_model["tipo"]}
URL: {data_model["url"]}
    """

    # Leer plantilla del prompt
    prompt_from_file = read_prompt_from_file(f"../src/data/input/prompts/knowledge_base_template.md")


    # Reemplazar variables
    first_research_prompt = replace_variables(prompt_from_file, {
        "{marca}": data_model["marca"],
        "{modelo}": data_model["modelo"],
        "{año}": str(data_model["año"]),
        "{pais}": COUNTRY,
        "{tipo}": data_model["tipo"]
        })

    deep_research = gemini_processor.send_prompt(first_research_prompt)

    knowledge_base = basic_data_model_info + "\n\n" + deep_research

    with open(f"../src/data/output/gemini/{get_current_date_format_yyyy_mm_dd()}-{data_model['code']}-{COUNTRY}.md", "w", encoding="utf-8") as f:
        f.write(knowledge_base)

IndexError: single positional indexer is out-of-bounds